In [ ]:
from datasets import load_dataset

# Load training data
train_dataset = load_dataset("wmt16", "de-en", split="train[:100]")

# Load validation data
validation_dataset = load_dataset("wmt16", "de-en", split="validation")

# Load test data
test_dataset = load_dataset("wmt16", "de-en", split="test")


/opt/homebrew/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load training data with IDs
from datasets import load_dataset, DatasetDict
train_dataset = load_dataset("wmt16", "de-en", split="train[:100]", with_id=True)

# Load validation data with IDs
validation_dataset = load_dataset("wmt16", "de-en", split="validation", with_id=True)

# Load test data with IDs
test_dataset = load_dataset("wmt16", "de-en", split="test", with_id=True)

german_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset,
})
print(german_dict)

In [ ]:
print(train_dataset)
print(train_dataset[0])

Dataset({
    features: ['translation'],
    num_rows: 100
})
{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    inputs = [ex for ex in examples['translation']]
    #just take the first translation
    inputs = [ex['de'] for ex in inputs]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    labu = [ex for ex in examples['translation']]
    labu = [ex['en'] for ex in labu]
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(labu, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
validation_dataset = validation_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}, 'input_ids': [15158, 24860, 74, 11216, 425, 7, 4267, 32, 221, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [419, 4078, 102, 1575, 13, 8, 2363, 1]}


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Make sure at least one layer is trainable
for param in model.parameters():
    param.requires_grad = True

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

trainer.train()


/opt/homebrew/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/26 [00:00<?, ?it/s]

ValueError: expected sequence of length 21 at dim 1 (got 57)

In [ ]:
from datasets import load_metric

bleu = load_metric("bleu")
meteor = load_metric("meteor")
bertscore = load_metric("bertscore")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Convert ids to tokens
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU, METEOR, and BERTScore here
    # Note: You will need to adjust this code to fit the metric computation as per the library's documentation

    return {
        "bleu": bleu_score,
        "meteor": meteor_score,
        "bertscore": bertscore_result,
    }

trainer.evaluate(eval_dataset=test_dataset)


In [ ]:
t

In [ ]:
import matplotlib.pyplot as plt

# Example code for plotting (you'll need to adapt this based on your actual logs)
plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Vs Epochs')
plt.legend()
plt.show()


In [ ]:
def translate(sentence, model, tokenizer):
    model.eval()
    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    output_ids = model.generate(input_ids)[0]
    return tokenizer.decode(output_ids, skip_special_tokens=True)

# Example use
translate("Wie geht es dir?", model, tokenizer)
